In [1]:
cd ..

/home/alexander/Учеба/ML/titanicH20


In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init();

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.20.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)
  Starting server from /home/alexander/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpehlq8bmj
  JVM stdout: /tmp/tmpehlq8bmj/h2o_alexander_started_from_python.out
  JVM stderr: /tmp/tmpehlq8bmj/h2o_alexander_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_alexander_ywbwzm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.902 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
import yaml

params = yaml.safe_load(open("params.yml"))

In [6]:
from src.data.make_dataset import load_data
from src.data.make_test import load_test

df, y = load_data(h2o, params['train_path'])
test = load_test(h2o, params['test_path'])

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
from src.models.train_model import train

aml = train(y, df, params)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
aml.leaderboard.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20201225_200240,0.873068,0.40305,0.849759,0.174672,0.351285,0.123401
XGBoost_grid__1_AutoML_20201225_200240_model_7,0.868986,0.420736,0.843071,0.173929,0.358614,0.128604
StackedEnsemble_BestOfFamily_AutoML_20201225_200240,0.868094,0.413276,0.846838,0.179609,0.357372,0.127715
XRT_1_AutoML_20201225_200240,0.866871,0.480681,0.834473,0.182629,0.361946,0.131005
XGBoost_grid__1_AutoML_20201225_200240_model_2,0.864975,0.44029,0.842475,0.178434,0.358761,0.128709
GBM_grid__1_AutoML_20201225_200240_model_9,0.864786,0.448233,0.847942,0.193438,0.367827,0.135297
GBM_grid__1_AutoML_20201225_200240_model_4,0.864738,0.43532,0.845206,0.198543,0.367933,0.135375
GBM_4_AutoML_20201225_200240,0.863926,0.437731,0.843665,0.191688,0.368904,0.13609
GBM_3_AutoML_20201225_200240,0.86346,0.4409,0.841415,0.202354,0.373592,0.139571
GBM_grid__1_AutoML_20201225_200240_model_8,0.863276,0.434044,0.839433,0.183779,0.365376,0.1335


In [9]:
from src.models.predict_model import predict

pred = predict(aml, test);

stackedensemble prediction progress: |████████████████████████████████████| 100%


/home/alexander/.local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Cabin' has levels not trained on: ["A11", "A18", "A21", "A29", "A9", "B10", "B11", "B24", "B26", "B36", "B45", "B52 B54 B56", "B61", "C105", "C116", "C130", "C132", "C28", "C31", "C39", "C51", "C53", "C55 C57", "C6", "C80", "C89", "C97", "D22", "D34", "D38", "D40", "D43", "E39 E41", "E45", "E52", "E60", "F", "F E46", "F E57"]
  warnings.warn(w)


In [10]:
from src.result.result import result

result(pred, test, params['reports_path'])